In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

c:\programacao\automacao\agentes-Langchain\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
class Sentimant(BaseModel):
    """Define o Sentimento e o idioma da mensagem enviada"""
    sentimento: str = Field(description="Sentimento do texto. Deve ser 'pos', 'neg', ou 'net' para neutro")
    lingua: str = Field(description="Lingua que o texto foi escrito (deve estar no charset UTF-8)")

tool_sentimento = convert_to_openai_function(Sentimant)
tool_sentimento

{'name': 'Sentimant',
 'description': 'Define o Sentimento e o idioma da mensagem enviada',
 'parameters': {'type': 'object',
  'properties': {'sentimento': {'description': "Sentimento do texto. Deve ser 'pos', 'neg', ou 'net' para neutro",
    'type': 'string'},
   'lingua': {'description': 'Lingua que o texto foi escrito (deve estar no charset UTF-8)',
    'type': 'string'}},
  'required': ['sentimento', 'lingua']}}

In [3]:
texto = "Eu gosto muito de pizza nordestina"

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

prompt = ChatPromptTemplate.from_messages([
    ("system", "Analise o texto e categorize-o conforme as instruções"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = prompt | chat.bind(functions = [tool_sentimento], 
                           function_call={"name": "Sentimant"})

In [6]:
chain.invoke({"input":texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"pos","lingua":"portuguese"}', 'name': 'Sentimant'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 135, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1cwBgWJqBbaNZqGy3trVdRl1G5fv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f8f18fb8-0267-4f3a-88b0-f349ea9c5435-0', usage_metadata={'input_tokens': 135, 'output_tokens': 13, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Categorização de texto

In [7]:
solicitacoes = [
    "Meu computador esta travando toda vez que tento abrir um programa. O que fazer",
    "Não consigo acessar minha conta. A senha não está funcionando",
    "Estou com dificuldades em instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar o endereço de entrega na plataforma",
    "O meu laptop não esta ligando. Acho que é um problema no cabo de força",
    "Não consigo ver meu arquivos no aplicativo. Existe alguma solução?"
]

In [8]:
from enum import Enum           
from langchain.pydantic_v1 import BaseModel, Field

class SetorEnum(str, Enum):
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros" 

class DirecionaSuporte(BaseModel):
    """Direciona a solicitação de suporte para o setor responsalvel"""
    setor: SetorEnum

In [9]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_direcionamento_suporte = convert_to_openai_function(DirecionaSuporte)
tool_direcionamento_suporte

{'name': 'DirecionaSuporte',
 'description': 'Direciona a solicitação de suporte para o setor responsalvel',
 'parameters': {'type': 'object',
  'properties': {'setor': {'description': 'An enumeration.',
    'enum': ['suporte_hardware',
     'suporte_software',
     'suporte_conta',
     'outros'],
    'type': 'string'}},
  'required': ['setor']}}

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento_suporte],
                            function_call={"name":"DirecionaSuporte"})
         | JsonOutputFunctionsParser())

In [12]:
solicitacao = solicitacoes[0]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Meu computador esta travando toda vez que tento abrir um programa. O que fazer
Resposta: {'setor': 'suporte_software'}


In [13]:
solicitacao = solicitacoes[3]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Gostaria de saber como atualizar o endereço de entrega na plataforma
Resposta: {'setor': 'suporte_conta'}


In [14]:
solicitacao = solicitacoes[4]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: O meu laptop não esta ligando. Acho que é um problema no cabo de força
Resposta: {'setor': 'suporte_hardware'}
